In [124]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns

<h2>Store some environment variables</h2>

In [125]:
import os

# Store the root directory of the project in a variable
CWD = os.getcwd() # path to notebook
BASE,_ = os.path.split(CWD) # on level up, the root directory 



print "Root project folder is: {}, notebooks is stored in {}".format(BASE,CWD)

Root project folder is: /Users/evanderknaap/Documents/Projects/tfvalidate, notebooks is stored in /Users/evanderknaap/Documents/Projects/tfvalidate/notebooks


<h3>Import data validation and try on a small dataset</h3>

In [38]:
# Import tensorflow
import tensorflow_data_validation as tfdv

# Get the working directory of the notebook. Then point to raw_data folder for the data 
#tfdv.generate_statistics_from_csv()

In [91]:
%%bash


bash: line 1: $TESTVAR: command not found
